In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=d48e3ee1d4b181001eb22cbb6f69034bdb0e9486d6848e451e17

In [ ]:
from nltk.cluster.util import cosine_distance
import re
import numpy as np
import pyidaungsu as pds
import pandas as pd

### Stop Words Function

In [ ]:
stopwordslist = []
slist = []
with open("stopwords.txt", encoding = 'utf8') as stopwordsfile:
    stopwords = stopwordsfile.readlines()
    slist.extend(stopwords)
    for w in range(len(slist)):
        temp = slist[w]
        stopwordslist.append(temp.rstrip())

In [ ]:
SENTENCE_SEPARATOR = [".", "?", "!", "။", "…", "\n"]

def sentence_tokenize(text):
  sentences = re.split("(?<=[" + "".join(SENTENCE_SEPARATOR) + "])\s*", text)
  if sentences[-1]:
    return sentences
  return sentences[:-1]

# Removed everything from the stopword list.
def stop_word(sentence):
  new_sentence = []
  for word in sentence:
    if word not in stopwordslist:
      new_sentence.append(word)
  return(new_sentence)

def read_text(text):
  sentences = sentence_tokenize(text)
  return sentences

def sentence_similarity(sent1, sent2):

  sent1 = [w.lower() for w in pds.tokenize(sent1, form='word')]
  sent2 = [w.lower() for w in pds.tokenize(sent2, form='word')]
  sent1 = stop_word(sent1)
  sent2 = stop_word(sent2)

  all_words = list(set(sent1 + sent2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  for w in sent1:
    vector1[all_words.index(w)] += 1

  for w in sent2:
    vector2[all_words.index(w)] += 1

  if np.isnan(1 - cosine_distance(vector1, vector2)):
    return 0
  return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences):

  similarity_matrix = np.zeros((len(sentences),len(sentences)))

  for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
      if idx1!=idx2:
        similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2])
    return similarity_matrix

In [ ]:
#@title Generate Text Summary
import networkx as nx

def generate_summary(text,top_n):

  summarize_text = []

  sentences = read_text(text)
  sentence_similarity_matrix = build_similarity_matrix(sentences)
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
  scores = nx.pagerank(sentence_similarity_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
  if not ranked_sentences:
        return ""  # Return empty summary and length 0
  else:
    for i in range(top_n):
      summarize_text.append(ranked_sentences[i][1].replace("\n", ""))
  return " ".join(summarize_text)



### Example 1

In [ ]:
text = """အေစီမီလန်အသင်း တိုက်စစ်ကစားသမား ခရစ္စတီယန်ပူလီဆစ်သည် ၂၀၂၃ ခုနှစ်အတွက် အမေရိကန်အကောင်းဆုံး ဘောလုံးသမားဆု ဆွတ်ခူးရရှိပြီး လေးကြိမ်ရရှိထားသူ ဒိုနိုဗန်၏စံချိန်ကို အမီလိုက်နိုင်ခဲ့ကြောင်း သိရသည်။ အသက် ၂၅ နှစ်ရှိပြီဖြစ်သူ အမေရိကန်လက်ရွေးစင် အသင်းခေါင်းဆောင် ပူလီဆစ်သည် ပြီးခဲ့သည့်နှစ်အတွင်း လက်ရွေးစင်အသင်းအတွက် ခုနစ်ပွဲကစား၊ ခြောက်ဂိုးသွင်းယူပေးပြီး ၂၀၂၂-၂၀၂၃ ကွန်ကာကပ်ဖ် Nations League ပြိုင်ပွဲ ၌ ဗိုလ်စွဲခဲ့သလို ယင်းပြိုင်ပွဲ၏ အကောင်းဆုံးကစားသမားဆုကို ဆွတ်ခူးရရှိခဲ့သည်။  ၎င်းအနေဖြင့် ချယ်လ်ဆီးအသင်း၌ ရုန်းကန်ခဲ့ရသော်လည်း ပြီးခဲ့သည့်နှစ် နွေရာသီတွင် အေစီမီလန်အသင်းသို့ ပြောင်းရွှေ့ခဲ့ပြီးနောက် ပုံမှန်ခြေစွမ်းပြန်လည်ရရှိလာပြီး ပြိုင်ပွဲစုံ ၂၆ ပွဲကစား၊ ခုနစ်ဂိုးသွင်းယူကာ ဂိုးဖန်တီးမှု ငါးကြိမ် ပြုလုပ်ပေးထားသည်။ ထို့ကြောင့် ပူလီဆစ်အနေဖြင့် မဲပေးမှု၏ ၅၃ ရာခိုင်နှုန်း အထိရရှိခဲ့ပြီး အေစီမီလန်အသင်းဖော်ကွင်းလယ်လူ မူဆာ (၂၁ ဒသမ ၅ ရာခိုင်နှုန်း)၊ PSV အသင်းတိုက်စစ်မှူး ရီကာဒို ပီပီ (၁၂ ဒသမ ၉ ရာခိုင်နှုန်း) တို့ထက် မဲအရေအတွက်အသာဖြင့် ဆွတ်ခူးရယူနိုင်ခဲ့ခြင်းဖြစ်သည်။  ပူလီဆစ်သည် ယင်းဆုကို စတုတ္ထအကြိမ် (၂၀၁၇၊ ၂၀၁၉၊ ၂၀၂၁၊ ၂၀၂၃) ရရှိခြင်းဖြစ်ပြီး ဂန္ထဝင်ကစားသမား လန်ဒန်ဒိုနိုဗန်၏ အကြိမ်အများဆုံးရရှိသည့်စံချိန်နှင့် တူညီသွားပြီဖြစ်သည်။"""
summary= generate_summary(text, 2)
print("Summary: ", summary)

Summary:  အေစီမီလန်အသင်း တိုက်စစ်ကစားသမား ခရစ္စတီယန်ပူလီဆစ်သည် ၂၀၂၃ ခုနှစ်အတွက် အမေရိကန်အကောင်းဆုံး ဘောလုံးသမားဆု ဆွတ်ခူးရရှိပြီး လေးကြိမ်ရရှိထားသူ ဒိုနိုဗန်၏စံချိန်ကို အမီလိုက်နိုင်ခဲ့ကြောင်း သိရသည်။ အသက် ၂၅ နှစ်ရှိပြီဖြစ်သူ အမေရိကန်လက်ရွေးစင် အသင်းခေါင်းဆောင် ပူလီဆစ်သည် ပြီးခဲ့သည့်နှစ်အတွင်း လက်ရွေးစင်အသင်းအတွက် ခုနစ်ပွဲကစား၊ ခြောက်ဂိုးသွင်းယူပေးပြီး ၂၀၂၂-၂၀၂၃ ကွန်ကာကပ်ဖ် Nations League ပြိုင်ပွဲ ၌ ဗိုလ်စွဲခဲ့သလို ယင်းပြိုင်ပွဲ၏ အကောင်းဆုံးကစားသမားဆုကို ဆွတ်ခူးရရှိခဲ့သည်။


### Example 2

In [ ]:
text = """တီကျစ်ကျောက် မီးသွေးဓာတ်အားပေး စက်ရုံကို သက်တမ်းတိုး ထပ်မံလည်ပတ် ရန် ရှမ်းပြည်နယ်အစိုးရနှင့် ၀န် ကြီးဌာနက ကန့်ကွက်ရန်မရှိ ကြောင်း လျှပ်စစ်နှင့် စွမ်းအင်၀န် ကြီးဌာန ပြောရေးဆိုခွင့်နှင့် သတင်းပြန်ကြားရေး အဖွဲ့က ဇူလိုင် ၁၂ ရက်တွင် သတင်းထုတ်ပြန်သည်။ ရှမ်းပြည်နယ်ရှိ ကျောက်မီး သွေးသုံး ဓာတ်အားပေးစက်ရုံကို ၂၀၁၇ မေလ ၂၆ ရက်တွင် စမ်း သပ်လည်ပတ်ရာ တစ်နှစ်ပြည့် ၍ ထပ်မံသက်တမ်းတိုးရန် စီစဉ် နေပြီး လက်ရှိ၌ ပြည်ထောင်စု အစိုးရအဖွဲ့နှင့် သမ္မတရုံးသို့ တင်ပြထားကြောင်း လျှပ်စစ်ဓာတ် အားထုတ်လုပ်ရေးလုပ်ငန်း ဦးဆောင်ညွှန်ကြားရေးမှူး ဦးခင်မောင်၀င်းက ပြောသည်။ ‘‘အရင်တုန်းက ပြာတွေ ထွက်တယ်။ မီးခိုးနံ့တွေ ထွက် တယ်။ ဟိုတုန်းကစက်တွေ ဟောင်းစုတ်နေပါပြီ။ အခုစက်တွေပြင် ပြီးသွားပြီ။ ကမ္ဘာ့ဘဏ်သတ်မှတ် ချက်နှုန်း ပတ်၀န်းကျင်ထိန်းသိမ်း ရေး ၀န်ကြီးဌာနက သတ်မှတ်ထားတဲ့ ဂိုက်လိုင်းအတိုင်း ဒီအတွင်းထဲ မှာ ပြင်ပြီးသွားပြီ’’ဟု ၎င်းကဆက် လက်ပြောသည်။ အဆိုပါ စက်ရုံကို တရုတ် ကုမ္ပဏီက စနစ်တကျပြုပြင်ကာ အမျိုးသား ပတ်၀န်းကျင်ဆိုင်ရာ အရည်အသွေး (ထုတ်လွှတ်မှု) လမ်းညွှန်ချက်များနှင့် ကမ္ဘာ့ဘဏ်၏ သတ်မှတ်ချက် စံချိန်စံညွှန်းများအတိုင်း ဆောင်ရွက်ပြီး ဖြစ်ကြောင်း ဦးခင်မောင်၀င်းက ပြောသည်။ အဆိုပါ တီကျစ်ကျောက်မီး သွေးစက်ရုံကို ၂၀၁၅ မေလက တရုတ် Wuxi Hua Guang လျှပ် စစ်စွမ်းအင် ကုမ္ပဏီက တင်ဒါ အောင်မြင်ထားပြီး ၂၀၁၇ မေလ ၁၆ ရက်တွင် ဒေသခံများ၏ သဘောတူညီမှုဖြင့် တစ်နှစ်စမ်း သပ်လည်ပတ်ခွင့်ရခဲ့သည်။ ထိုစက်ရုံတွင် လျှပ်စစ်ဓာတ် အား စက်တပ်ဆင်အား မဂ္ဂါ၀ပ် ၁၂၀ ရှိပြီး လက်ရှိ၌ မဂ္ဂါ၀ပ် ၈၀ ထုတ်လုပ်နေသည်။ တရုတ် ကုမ္ပဏီ၏ လူမှုအကျိုးပြု အစီအ စဉ်ဖြင့် စက်ရုံပတ်လည်ရှိ ကျေး ရွာရှစ်ရွာတွင် လျှပ်စစ်မီးလင်းရေး ဆောင်ရွက်ခဲ့သည်ဟု ၀န်ကြီးဌာနက ဆိုထားသည်။ ထို့ကြောင့် ကျေးရွာများရှိ ဒေသခံအများစုက စက်ရုံကို ကန့်ကွက်မှုမရှိကြောင်း ဦးခင်မောင် ၀င်းက ဆိုသည်။"""
summary= generate_summary(text, 3)
print("Summary: ", summary)

Summary:  တီကျစ်ကျောက် မီးသွေးဓာတ်အားပေး စက်ရုံကို သက်တမ်းတိုး ထပ်မံလည်ပတ် ရန် ရှမ်းပြည်နယ်အစိုးရနှင့် ၀န် ကြီးဌာနက ကန့်ကွက်ရန်မရှိ ကြောင်း လျှပ်စစ်နှင့် စွမ်းအင်၀န် ကြီးဌာန ပြောရေးဆိုခွင့်နှင့် သတင်းပြန်ကြားရေး အဖွဲ့က ဇူလိုင် ၁၂ ရက်တွင် သတင်းထုတ်ပြန်သည်။ ရှမ်းပြည်နယ်ရှိ ကျောက်မီး သွေးသုံး ဓာတ်အားပေးစက်ရုံကို ၂၀၁၇ မေလ ၂၆ ရက်တွင် စမ်း သပ်လည်ပတ်ရာ တစ်နှစ်ပြည့် ၍ ထပ်မံသက်တမ်းတိုးရန် စီစဉ် နေပြီး လက်ရှိ၌ ပြည်ထောင်စု အစိုးရအဖွဲ့နှင့် သမ္မတရုံးသို့ တင်ပြထားကြောင်း လျှပ်စစ်ဓာတ် အားထုတ်လုပ်ရေးလုပ်ငန်း ဦးဆောင်ညွှန်ကြားရေးမှူး ဦးခင်မောင်၀င်းက ပြောသည်။ ထို့ကြောင့် ကျေးရွာများရှိ ဒေသခံအများစုက စက်ရုံကို ကန့်ကွက်မှုမရှိကြောင်း ဦးခင်မောင် ၀င်းက ဆိုသည်။


### Example 3

In [ ]:
text = """IS ပိုင် ရေနံတွင်းတွေကို ဗြိတိသျှ တိုက်လေယာဉ် ၄ စီးက တိုက်ခိုက်ခဲ့ ဗုံးကြဲ တိုက်ခိုက်ဖို့ အတွက် ယူကေ အမတ်တွေ အတည်ပြုပြီး မကြာမီမှာပဲ ဆိုက်ပရပ်စ်မှာ ရှိတဲ့ Akrotiri အက်ရော့တီရီ လေတပ်စခန်းက တိုနေးဒိုး တိုက်လေယာဉ် ၄ စီး တိုက်ခိုက်မှုမှာ ပါဝင် ခဲ့ပါတယ်။ ဆီးရီးယား အရှေ့ပိုင်းက IS တွေ ထိန်းချုပ်ထားတဲ့ ရေနံတွင်းတွေကို တိုက်ခိုက် ထိမှန်ခဲ့တယ်လို့ ကာကွယ်ရေး ဝန်ကြီးက ပြောပါတယ်။ ဝန်ကြီးချုပ် ဒေးဗစ် ကင်မရွန်း ကတော့ IS အပေါ် တိုက်ခိုက်မှုတွေဟာ အချိန် ကြာမြင့်မှာ ဖြစ်ပြီး ပုံမှန် ပြုလုပ်သွားဖို့ လိုတယ်လို့ သတိပေး ပြောဆို ခဲ့ပါတယ်။ IS အပေါ် လေကြောင်း တိုက်ဖို့အတွက် ယူကေ ပါလီမန်မှာ ဗုဒ္ဓဟူးနေ့က ဆွေးနွေးငြင်းခုံပြီး မဲခွဲ ဆုံးဖြတ်ရာမှာ ထောက်ခံမဲ ၃၉၇ မဲ၊ ကန့်ကွက်မဲ ၂၂၃ မဲနဲ့ အတည်ပြုခဲ့တာ ဖြစ်ပါတယ်။ အက်ရော့တီရီ လေတပ်စခန်းမှာ ရှိနှင့်ပြီးဖြစ်တဲ့ တိုက်လေယာဉ် ၈ စီး အပြင် နောက်ထပ် ၈ စီး ထပ်ပို့လိုက်တယ် လို့လည်း ယူကေ ကာကွယ်ရေး ဝန်ကြီး မိုက်ကယ် ဖာလန်က ပြောပါတယ်။"""
summary= generate_summary(text, 2)
print("Summary: ", summary)

Summary:  IS ပိုင် ရေနံတွင်းတွေကို ဗြိတိသျှ တိုက်လေယာဉ် ၄ စီးက တိုက်ခိုက်ခဲ့ ဗုံးကြဲ တိုက်ခိုက်ဖို့ အတွက် ယူကေ အမတ်တွေ အတည်ပြုပြီး မကြာမီမှာပဲ ဆိုက်ပရပ်စ်မှာ ရှိတဲ့ Akrotiri အက်ရော့တီရီ လေတပ်စခန်းက တိုနေးဒိုး တိုက်လေယာဉ် ၄ စီး တိုက်ခိုက်မှုမှာ ပါဝင် ခဲ့ပါတယ်။ ဆီးရီးယား အရှေ့ပိုင်းက IS တွေ ထိန်းချုပ်ထားတဲ့ ရေနံတွင်းတွေကို တိုက်ခိုက် ထိမှန်ခဲ့တယ်လို့ ကာကွယ်ရေး ဝန်ကြီးက ပြောပါတယ်။



## Comparison between Humman Summary and Result
Summary: ဗြိတိသျှ တော်ဝင် လေတပ်ရဲ့ တိုနေဒိုး
တိုက်လေယာဉ်တွေက ဆီးရီးယားမှာ ရှိတဲ့ IS အစ္စလာမ္မစ် နိုင်ငံအဖွဲ့ အပေါ် လေကြောင်း တိုက်ခိုက်မှုတွေ လုပ်ခဲ့တယ်လို့ ယူကေ ကာကွယ်ရေး ဝန်ကြီးဌာနက အတည်ပြု ပြောဆိုခဲ့ပါတယ်။

Output: IS ပိုင် ရေနံတွင်းတွေကို ဗြိတိသျှ တိုက်လေယာဉ် ၄ စီးက တိုက်ခိုက်ခဲ့ ဗုံးကြဲ တိုက်ခိုက်ဖို့ အတွက် ယူကေ အမတ်တွေ အတည်ပြုပြီး မကြာမီမှာပဲ ဆိုက်ပရပ်စ်မှာ ရှိတဲ့ Akrotiri အက်ရော့တီရီ လေတပ်စခန်းက တိုနေးဒိုး တိုက်လေယာဉ် ၄ စီး တိုက်ခိုက်မှုမှာ ပါဝင် ခဲ့ပါတယ်။ ဆီးရီးယား အရှေ့ပိုင်းက IS တွေ ထိန်းချုပ်ထားတဲ့ ရေနံတွင်းတွေကို တိုက်ခိုက် ထိမှန်ခဲ့တယ်လို့ ကာကွယ်ရေး ဝန်ကြီးက ပြောပါတယ်။

End of the prject!!